# Replicating mini EPSCs and location-dependent membrane filtering for synaptic inputs

In [3]:
%matplotlib notebook

import sys
import csv

from neuron import h, gui
from neuron.units import ms, mV

from matplotlib import pyplot
import numpy as np
import pandas as pd

# allow importing module from parent folder
sys.path.append('../')

from synaptic_stimulation import *

%load_ext autoreload
%autoreload 2

## Import Bokeh for graphs

In [4]:
import bokeh.plotting as plt
from bokeh.io import output_notebook, export_svg, export_png
from bokeh.models import Legend, LegendItem, LabelSet, ColumnDataSource
from bokeh.models.ranges import DataRange1d
from bokeh.models.tools import CrosshairTool, ExamineTool

output_notebook()

Loading BokehJS ...

## Load morphology and other components

In [5]:
# load channels
h('''{load_file("../chan-DmNaT-ODowd.hoc")
      load_file("../chan-DmNaP-DmNav10.hoc")
      load_file("../chan-DmKdr-Marley.hoc")
      load_file("../chan-DmKA-Marley.hoc")
     }''')

# load morphology
h('{load_file("../neuron-morph-ext-axon-2pieces-chans-ext-axon-70um.hoc")}')

# "fix" reversals in active section
h.axon_ext[1].ena = 45
h.axon_ext[1].ek = -85
h.psection(sec=h.axon_ext[1])

# enable or disable voltage clamp
h('load_file("../vclamp_soma_-60mV.ses")')

	0 
	0 
	0 
	0 
	0 
axon_ext[1] { nseg=25  L=500  Ra=189
	axon_ext[0] connect axon_ext[1] (0), 1
	/* First segment only */
	insert DmNaP { gmax_DmNaP=0.00011}
	insert DmNaT { gmax_DmNaT=0.24}
	insert na_ion { ena=45}
	insert DmKdr { gmax_DmKdr=0.85}
	insert k_ion { ek=-85}
	insert pas { g_pas=0.00012 e_pas=-85}
	insert capacitance { cm=0.7}
	insert morphology { diam=0.7}
}
	1 


1

## Initialization

In [6]:
# load common funcs
h('load_file("../fitfuncs.hoc")')

#h('print_elec_cell()') #fix this 

# calc morph stats
h('load_file("../stats.hoc")')

file_name = "ext-axon-70um-sim-synapses"

# save state in this file
h.state_file_name = "state-neuron-act+elec+ext-axon-2piece-chans-" + file_name + ".bin"

# small adjustments

# increase VC duration
h.VClamp[0].dur[0] = 1000

	1 
Secname	L (um)	Diam (um)	Area [um^2]	Ri [MO]		Cm [pF]	g_pas [nS]
axon:	37.61	1.14	134.11	35.12	1.34	0.00
soma:	6.98	5.50	120.56	0.28	1.21	0.00
bottom dend:	996.91	0.78	2430.62	1991.49	24.31	0.02
botdend - axon:	959.10	0.76	2292.50	1993.51	22.92	0.02
topdend:	366.00	0.73	837.85	829.37	8.38	0.01
ext. axon:	570.00	0.71	1275.49	1351.81	12.75	1.53
	1 


In [7]:
'''
 * synapses:
 * top dend: 685, 524, 520, 626
 * bot dend: 205, 357, 464, 588, 513, 48
'''

create_syns(h, h.dendrite[685], h.dendrite[524], h.dendrite[520], h.dendrite[626], h.dendrite[205], h.dendrite[357], 
                h.dendrite[464], h.dendrite[588], h.dendrite[513], h.dendrite[48], # end of the 10 distal positions
                h.soma[0], h.dendrite[331], h.dendrite[271] # close to soma
                )
#set_syn_pars()                  # use defaults

Creating synapse on dendrite[685]
Creating synapse on dendrite[524]
Creating synapse on dendrite[520]
Creating synapse on dendrite[626]
Creating synapse on dendrite[205]
Creating synapse on dendrite[357]
Creating synapse on dendrite[464]
Creating synapse on dendrite[588]
Creating synapse on dendrite[513]
Creating synapse on dendrite[48]
Creating synapse on soma[0]
Creating synapse on dendrite[331]
Creating synapse on dendrite[271]


# Experiments

## create shape plot

In [8]:
ps = h.PlotShape(0) # don't show a default view
ps.show(0)         # show diameters
ps.view(30,10,55,87, 300, 100, 500, 700)
h.load_file('stdrun.hoc')

1.0

In [9]:
vc_current = h.Vector().record(h.VClamp[0]._ref_i)  # Voltage clamp current vector
v_soma = h.Vector().record(h.soma[0](0.5)._ref_v)  # Membrane potential vector
t = h.Vector().record(h._ref_t)  # Time stamp vector

## Run a short simulation to save steady state

In [ ]:
# warning! if state changes (new synapses, etc) restoring saved state file will cause a crash!
h.finitialize(-60 * mV)
h.continuerun(100 * ms)
# rewind and save
h.finitialize()
h.saveState()
print("State saved to: " + h.state_file_name)

## Stimulate one synapse (dendrite 685, #1) with varying magnitudes to check changes in rise and fall times that explain observed variability

Observed rise times vary between 0-9 ms and fall times vary between 0-30 ms.

In [10]:
syn = syns['dendrite[685]']

Synapses 685 is on the tip of the contralateral dendrite, but electrotonically not so far from the soma according to the 2015 paper passive/active parameters.

In [11]:
h.restoreState()
syn.set_stim(ps = ps, weight = .000001, taurise = 1, taufall=5, interval = 0, number = 1)
h.finitialize()
h.continuerun(60 * ms)

Reading state from file.
Setting: weight=1e-06, taurise = 1, taufall=5, interval=0, magepsc=20, syne=0, number=1, start=10


0.0

In [12]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="VC current (nA)")
f.add_tools(CrosshairTool())
f.line(np.array(t), np.array(vc_current), line_width=2)
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -0.01, end = 0)
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-685.svg")

### Extract information

In [13]:
extract_mini_metrics(h, np.array(vc_current), 10)

{'risetime_ms': 3.2,
 'falltime_ms': 25.275000000000002,
 'amp_pA': -1.0387280346435546}

### Run a sweep of amplitudes

In [14]:
vc_currents_685, list_steps_dend685 = run_amp_sweep(h, ps, vc_current, syn, .0001, 0.001, 6)

Simulating step of synaptic weight of 0.0001.
Reading state from file.
Setting: weight=0.0001, taurise = 1, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00015848931924611142.
Reading state from file.
Setting: weight=0.00015848931924611142, taurise = 1, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00025118864315095795.
Reading state from file.
Setting: weight=0.00025118864315095795, taurise = 1, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00039810717055349735.
Reading state from file.
Setting: weight=0.00039810717055349735, taurise = 1, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.000630957344480193.
Reading state from file.
Setting: weight=0.000630957344480193, taurise = 1, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic wei

In [ ]:
# no need with autoreload?
import importlib
importlib.reload(sys.modules['synaptic_stimulation'])

In [15]:
f = plot_amp_sweep(t, vc_currents_685, "w=[" + " ".join("{0:.4f} ".format(num) for num in list_steps_dend685) + "]")
f.x_range = DataRange1d(start = 9, end = 40)
f.y_range = DataRange1d(start = -55, end = 1)
f.frame_width = 400
f.frame_height = 300
f.legend.location = 'bottom_right'
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-685-amp-sweep.svg")

In [ ]:
minis_den685_df = get_sweep_metrics(h, vc_currents_685, list_steps_dend685)
plot_scatter_risetime_amp(minis_den685_df)
minis_den685_df

Although fall time increased with varying amplitudes, there was no discernible change in the rise time. So rise time is not affected by stimulus amplitude (find refs to back this up). 685 is on the contralateral dendrite and closer to the soma, so we expect more delays from the ipsilateral dendrite.

## Synapse #8 (ipsilateral), electrotonically furthest from soma

In [ ]:
syn2 = syns['dendrite[588]']

In [ ]:
#syn.set_stim(ps = ps, weight = 0, tau=5, interval = 0, number = 1)
h.restoreState()
#syn.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
syn2.set_stim(ps = ps, weight = .00018, taurise=1, taufall=5, interval = 0, number = 1)
h.finitialize()
h.continuerun(60 * ms)

In [ ]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="VC current (nA)")
f.add_tools(CrosshairTool())
f.line(np.array(t), np.array(vc_current), line_width=2)
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -0.10, end = 0)
plt.show(f)

In [ ]:
vc_currents_588, steps_588 = run_amp_sweep(h, ps, vc_current, syn2, .0001, 0.001, 6)

In [ ]:
f = plot_amp_sweep(t, vc_currents_588, "w=[" + " ".join("{0:.4f} ".format(num) for num in steps_588) + "]")
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-588-amp-sweep.svg")

In [ ]:
minis_den588_df = get_sweep_metrics(h, vc_currents_588, steps_588)
plot_scatter_risetime_amp(minis_den588_df)
minis_den588_df.index.name = "syn_weight"
minis_den588_df

Same issue as 685: rise time does not change with stimulus amplitude. Increasing synapse $\tau_{rise}$ parameter also inreases observed rise time, but does not increase its variance.

## Sweep Ra values and measure rise time

Deafult value of $R_a$ from Gunay et al (2015) was 189. Gouwens and Wilson (2009) reported fits to different cells a range of 102.5 to 266.1 for central olfactory neurons (antennal lobe projection neurons).

In [ ]:
def set_Ra(amp):
    print(f"Simulating Ra = {amp}.")
    h.set_Ra(amp)
    syn2.set_stim(ps = ps, weight = .00018, taurise=1, taufall=5, interval = 0, number = 1)

Ra_steps = [100, 200, 500, 750, 1000]
vc_currents_Ra = run_sweep_func(h, ps, vc_current, set_Ra, Ra_steps)

# reset g_pas back to default
h.set_Ra(185)

In [ ]:
f = plot_amp_sweep(t, vc_currents_Ra, 'Ra = ' + str(Ra_steps))
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -140, end = 10)
f.frame_width = 300
f.frame_height = 200
f.legend.location = 'bottom_right'
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-588-Ra-sweep.svg")

In [ ]:
minis_den588_Ra_df = get_sweep_metrics(h, vc_currents_Ra, Ra_steps)
minis_den588_Ra_df.index.name = "Ra"
minis_den588_Ra_df

In [ ]:
f = plt.figure(x_axis_label=r"$$R_a~(\Omega\cdot \mathrm{cm})$$", y_axis_label="rise time (ms)", frame_width = 400)
f.add_tools(CrosshairTool())
f.line(x='Ra', y='risetime_ms', source=minis_den588_Ra_df, alpha=1)
f.circle(x='Ra', y='risetime_ms', source=minis_den588_Ra_df, alpha=1)
f.frame_width = 200
f.frame_height = 200
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="compare-scatter-Ra-sweeps-dendrite-588.svg")

In [ ]:
# save as PNG as well because SVG exports don't show LaTeX labels properly outside of browser (Inkscape, LyX)
export_png(f, filename="compare-scatter-Ra-sweeps-dendrite-588.png")

## Sweep leak conductance values and measure rise time

Deafult value of $g_pas$ from Gunay et al (2015) was a specific leak conductance of 10^-6 S/cm^2 (or 1 MOhm.cm^2). Gouwens and Wilson (2009) reported fits to different cells a range of 8-20 kOhms.cm^2 for central olfactory neurons (antennal lobe projection neurons).

In [ ]:
def set_g_pas(amp):
    print(f"Simulating g_pas = {amp}.")
    set_g_pas_dend(h, amp)
    syn2.set_stim(ps = ps, weight = .00018, taurise=1, taufall=5, interval = 0, number = 1)

g_pas_steps = np.logspace(np.log10(1e-6), np.log10(.1), 6)
vc_currents_gPas = run_sweep_func(h, ps, vc_current, set_g_pas, g_pas_steps)

# reset g_pas back to default
set_g_pas_dend(h, 1e-6)

In [ ]:
f = plot_amp_sweep(t, vc_currents_gPas)
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -140, end = 950)
plt.show(f)

In [ ]:
minis_den588_gPas_df = get_sweep_metrics(h, vc_currents_gPas, g_pas_steps)
minis_den588_gPas_df.index.name = "g_pas"
minis_den588_gPas_df

As expected, varying $g_\mathrm{pas}$ do not affect synaptic rise time.

## Ipsilateral, central synapse #10

This synapse is closer to the soma as it's central-ipsilateral location in the dendritic field.

In [ ]:
syn3 = syns['dendrite[48]']

In [ ]:
vc_currents_48, steps_48 = run_amp_sweep(h, ps, vc_current, syn3, .0001, 0.002, 6)

In [ ]:
f = plot_amp_sweep(t, vc_currents_48, "w=[" + " ".join("{0:.4f} ".format(num) for num in steps_48) + "]")
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-48-amp-sweep.svg")

In [ ]:
minis_den48_df = get_sweep_metrics(h, vc_currents_48, steps_48)
plot_scatter_risetime_amp(minis_den48_df)
minis_den48_df

## Synapse at soma

In [ ]:
syn4 = syns['soma[0]']

In [ ]:
vc_currents_soma, steps_soma = run_amp_sweep(h, ps, vc_current, syn4, .0001, 0.001, 6)

In [ ]:
f = plot_amp_sweep(t, vc_currents_soma, "w=[" + " ".join("{0:.4f} ".format(num) for num in steps_soma) + "]")
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-48-amp-sweep.svg")

In [ ]:
minis_soma0_df = get_sweep_metrics(h, vc_currents_soma, steps_soma)
plot_scatter_risetime_amp(minis_soma0_df)
minis_soma0_df

# Compare metrics

In [ ]:
f = plt.figure(x_axis_label="rise time (ms)", y_axis_label="amplitude (pA)", frame_width = 400, frame_height = 400)
f.add_tools(CrosshairTool())
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_soma0_df, color='purple', legend_label='soma0')
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den685_df, color='red', legend_label='dend685 (#1)')
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den48_df, color='green', legend_label='dend48 (#10)') 
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den588_df, color='blue', legend_label='dend588 (#8)')
#f.x_range = DataRange1d(start = 0.8, end = 1.1)
#f.y_range = DataRange1d(start = -70, end = 0)
f.legend.location = 'bottom_left'
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="compare-scatter-amp-sweeps.svg")

### Plot only first step that is same and compare amplitudes

In [ ]:
syn_df = pd.DataFrame()
names = ["soma", "dend685 (#1)", 'dend48 (#10)', 'dend588 (#8)']
for _df in [minis_soma0_df, minis_den685_df, minis_den48_df, minis_den588_df]:
    _df.attrs["name"] = names.pop(0)
    syn_df[_df.attrs["name"]] = _df.rename_axis('syn_w').reset_index().iloc[0,:]
syn_df = syn_df.T
#syn_df = syn_df.rename_axis('location').reset_index()
syn_df

In [ ]:
f = plt.figure(x_axis_label="rise time (ms)", y_axis_label="amplitude (pA)", frame_width = 300, frame_height = 300)
f.add_tools(CrosshairTool())
#f.vbar(top='amp_pA',  source=syn_df)
f.line(x='risetime_ms', y='amp_pA',  source=syn_df)
labels = LabelSet(x='risetime_ms', y='amp_pA', text='location',
                  x_offset=0, y_offset=-10, source=ColumnDataSource(syn_df), angle_units='deg', angle=-30)
f.add_layout(labels)
#f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den685_df, color='red', legend_label='dend685 (#1)')
#f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den48_df, color='green', legend_label='dend48 (#10)') 
#f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den588_df, color='blue', legend_label='dend588 (#8)')
f.x_range = DataRange1d(start = 2.5, end = 5.5)
f.y_range = DataRange1d(start = -70, end = 0)
plt.show(f)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="compare-amp-vs-risetime-syns-w-0.0001.svg")

## To Do
- add some figures here from other documents to increase readability 
- simulate what does halving na does to reversal and excitability + change mini parameters in SRC reproduction code to mimic recordings
- rerun passive fits by increasing values for Ra (only for dendrites) and see if it works (not so much needed because of limits of light microscopy we can't model synapses on thinner processes)
- make plot of distance while keeping amplitude constant
- analyze new half-Na SRC data to see if lowering frequency explains SRC size; analyze some of the new files

## Scrapbook

In [ ]:
minis_den588_Ra_100_df.index = pd.Index([189])

In [ ]:
h.axon_ext[1].ena = 45
h.axon_ext[1].ek = -85
h.psection(sec=h.axon_ext[1])

In [ ]:
h.restoreState()
for sec in h.ext_axonic: 
    print(sec.g_pas)

In [ ]:
h.set_Ra(185)
h.soma[0].Ra